In [17]:
!pip -q install -U \
  langchain==0.2.16 \
  langchain-core==0.2.41 \
  langchain-community==0.2.16 \
  langchain-google-genai==1.0.10 \
  faiss-cpu newspaper3k textblob tiktoken wikipedia duckduckgo-search sentence-transformers lxml_html_clean


In [ ]:
import os
os.kill(os.getpid(), 9)


In [10]:
import os, time, hashlib
from typing import Dict, Any

from newspaper import Article
from textblob import TextBlob

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document


In [11]:
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) NewsSenseAI/1.0"

print("✅ Keys loaded")


✅ Keys loaded


In [12]:
# ✅ Models in fallback order (most quota-friendly first)
MODEL_FALLBACK = [
    "models/gemini-flash-lite-latest",
    "models/gemini-2.0-flash-lite",
    "models/gemini-2.0-flash",
    "models/gemini-pro-latest"
]

# ✅ RAG configuration (token efficient)
CHUNK_SIZE = 600
CHUNK_OVERLAP = 100
TOP_K = 2  # keep low to reduce tokens

# ✅ In-memory cache to avoid repeated API calls
CACHE: Dict[str, Any] = {}


In [13]:
def make_cache_key(*parts) -> str:
    raw = "||".join([str(p) for p in parts])
    return hashlib.md5(raw.encode("utf-8")).hexdigest()

def load_article_text(url: str):
    """
    BBC-safe article extraction
    """
    art = Article(url)
    art.download()
    art.parse()
    title = art.title.strip()
    text = art.text.strip()
    return title, text

def sentiment_offline(text: str) -> Dict[str, float]:
    """
    Offline sentiment with TextBlob
    """
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        tone = "Positive 😊"
    elif polarity < -0.1:
        tone = "Negative 😠"
    else:
        tone = "Neutral 😐"
    return {"tone": tone, "polarity": float(polarity)}


In [14]:
def gemini_call(prompt: str, model_name: str, temperature=0):
    llm = ChatGoogleGenerativeAI(model=model_name, temperature=temperature)
    return llm.invoke(prompt).content

def safe_gemini(prompt: str, cache_id: str, temperature=0, max_retries=6):
    """
    Optimal Gemini caller:
    ✅ caching
    ✅ fallback models
    ✅ retry with exponential backoff
    """
    if cache_id in CACHE:
        return CACHE[cache_id]

    last_err = None

    for model_name in MODEL_FALLBACK:
        delay = 5
        for attempt in range(max_retries):
            try:
                out = gemini_call(prompt, model_name=model_name, temperature=temperature)
                CACHE[cache_id] = out
                return out
            except Exception as e:
                last_err = e
                msg = str(e)

                # rate limit / quota
                if ("429" in msg) or ("ResourceExhausted" in msg):
                    print(f"⚠️ 429 rate limit on {model_name}. Sleeping {delay}s...")
                    time.sleep(delay)
                    delay = min(delay * 2, 60)
                    continue

                # model not supported
                if ("404" in msg) or ("not found" in msg):
                    print(f"⚠️ Model not supported: {model_name}. Trying next model...")
                    break

                # other error
                break

    raise RuntimeError(f"❌ Gemini failed after retries/models. Last error: {last_err}")


In [15]:
def build_vectorstore(article_text: str):
    docs = [Document(page_content=article_text)]
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP
    )
    splits = splitter.split_documents(docs)

    emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vs = FAISS.from_documents(splits, emb)
    return vs


In [16]:
def summarize_article(vectordb, url: str) -> str:
    """
    One-shot RAG summary:
    ✅ retrieve top chunks
    ✅ single Gemini call
    """
    # retrieve once (no Gemini here)
    docs = vectordb.similarity_search("Give a concise summary of the article", k=TOP_K)
    context = "\n\n".join([d.page_content for d in docs])

    prompt = f"""
You are NewsSense AI.
Summarize the following news article using ONLY the context.

Rules:
- EXACTLY 3 bullet points
- Each bullet must be max 2 lines
- No extra headings/text

CONTEXT:
{context}
"""
    cache_id = make_cache_key(url, "summary", TOP_K, CHUNK_SIZE)
    return safe_gemini(prompt, cache_id)

def extract_claims(vectordb, url: str) -> str:
    """
    Optional claim extraction: 1 Gemini call
    """
    docs = vectordb.similarity_search("Extract fact-checkable claims", k=TOP_K)
    context = "\n\n".join([d.page_content for d in docs])

    prompt = f"""
Extract 5 fact-checkable claims from the news context below.
Rules:
- Output ONLY numbered list (1..5)
- Each claim short & verifiable

CONTEXT:
{context}
"""
    cache_id = make_cache_key(url, "claims", TOP_K, CHUNK_SIZE)
    return safe_gemini(prompt, cache_id)


In [17]:
def ask_question(vectordb, url: str, question: str) -> str:
    docs = vectordb.similarity_search(question, k=TOP_K)
    context = "\n\n".join([d.page_content for d in docs])

    prompt = f"""
Answer the question using ONLY the context.
If the answer is not in context, say: "Not found in article."

CONTEXT:
{context}

QUESTION:
{question}
"""
    cache_id = make_cache_key(url, "qna", question, TOP_K, CHUNK_SIZE)
    return safe_gemini(prompt, cache_id)


In [18]:
def analyze_news(url: str, include_claims=False) -> Dict[str, Any]:
    """
    Main pipeline:
    ✅ article load
    ✅ vector index
    ✅ 1-call summary
    ✅ offline sentiment
    ✅ optional claims
    """
    print(f"\n🔄 Processing URL: {url}")

    # Article load cached (avoid re-downloading)
    cache_id = make_cache_key(url, "article")
    if cache_id in CACHE:
        title, text = CACHE[cache_id]
    else:
        title, text = load_article_text(url)
        CACHE[cache_id] = (title, text)

    if len(text) < 400:
        return {"error": "❌ Article text too short / blocked"}

    print("✅ Article loaded:", title)

    # Vectorstore cached
    vs_key = make_cache_key(url, "vectorstore", CHUNK_SIZE, CHUNK_OVERLAP)
    if vs_key in CACHE:
        vectordb = CACHE[vs_key]
    else:
        vectordb = build_vectorstore(text)
        CACHE[vs_key] = vectordb
        print("✅ Indexed: FAISS ready")

    # Summary (1 Gemini call)
    summary = summarize_article(vectordb, url)

    # Sentiment offline
    senti = sentiment_offline(text[:3000])

    output = {
        "title": title,
        "summary": summary,
        "sentiment": senti,
        "vectorstore": vectordb  # keep for QnA
    }

    # Optional claims (1 Gemini call)
    if include_claims:
        output["claims"] = extract_claims(vectordb, url)

    return output


In [19]:
url_to_test = "https://www.bbc.com/news/articles/cn5ln4wee7wo"

res = analyze_news(url_to_test, include_claims=False)

print("\n" + "="*60)
print("📰 NEWS REPORT")
print("="*60)
print("TITLE:", res.get("title"))

print("\n✅ SUMMARY:\n", res.get("summary"))
print("\n✅ SENTIMENT:\n", res.get("sentiment"))
print("="*60)



🔄 Processing URL: https://www.bbc.com/news/articles/cn5ln4wee7wo
✅ Article loaded: Aravalli: Why India's ancient hills are at the centre of growing protests
✅ Indexed: FAISS ready

📰 NEWS REPORT
TITLE: Aravalli: Why India's ancient hills are at the centre of growing protests

✅ SUMMARY:
 * Activists demand the government define Aravalli areas using scientific criteria like geography and ecology.
* A critical role in ecology or preventing desertification should be recognized as part of the range, regardless of height.
* Critics warn the court's new definition could encourage mining, construction, and commercial activity, risking ecological damage.

✅ SENTIMENT:
 {'tone': 'Neutral 😐', 'polarity': 0.07667189132706373}


In [26]:
if "vectorstore" in res:
    q = "where the protest is happening?"
    ans = ask_question(res["vectorstore"], url_to_test, q)

    print("\n" + "="*60)
    print("🤖 RAG Q&A")
    print("="*60)
    print("Q:", q)
    print("A:", ans)
    print("="*60)



🤖 RAG Q&A
Q: where the protest is happening?
A: Gurugram city near Delhi
